In [4]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# import pandas as pd
# import numpy as nppip
# import matplotlib.pyplot as plt
# %matplotlib inline

# # Detect hardware, return appropriate distribution strategy
# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
#     print('Running on TPU ', tpu.master())
# except ValueError:
#     tpu = None

# if tpu:
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.experimental.TPUStrategy(tpu)
# else:
#     strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

# print("REPLICAS: ", strategy.num_replicas_in_sync)

In [5]:
#3. Read file as panda dataframe
import pandas as pd  # 数据分析

data = pd.read_csv("../input/guangzhou/.csv")
data

In [6]:
data.info()

In [7]:
data.head()

In [8]:
data.tail()

In [9]:
data.columns

In [10]:
data['aqi'].isna().sum()

In [11]:
data=data.set_index('ymd')
data

In [12]:
data

In [13]:
# # del data['city_list']
# # del data['Unnamed: 0']
# del data['aqiInfo']
# del data['aqiLevel']
# data

将data的tianqi+fengxiang+fengli+bWendu+yWendu数值化（独热编码）

In [14]:
tianqi=data.tianqi.unique()
tianqi_to_index={}
j=1
for i in set(tianqi):
    tianqi_to_index[i]=j
    j+=1
tianqi_to_index

In [15]:
data['tianqi']=data.apply(lambda x:tianqi_to_index[x['tianqi']],axis=1)
data

In [16]:
del data['city_list']
del data['Unnamed: 0']
del data['aqiInfo']
del data['aqiLevel']
data

In [17]:
fengli=data.fengli.unique()
fengli

In [18]:
s=set(fengli)
s

In [19]:
j=1
fengli_to_index={}
for i in s:
    fengli_to_index[i]=j
    j+=1
fengli_to_index['微风']=0
fengli_to_index

In [20]:
# for i in data['fengli']:
#     i=fengli_to_index[i]
# data
data['fengli']=data.apply(lambda x:fengli_to_index[x['fengli']],axis=1)
data

## 现在操作风向


In [21]:
data.fengxiang.unique()

In [22]:
data=data.join(pd.get_dummies(data.fengxiang))
del data['fengxiang']
data

## 现在操作温度

In [23]:
# ℃
t1=data['bWendu'].unique()
t1=[x.split('℃')[0] for x in t1 ]
t1

In [24]:
j=1
t1_to_index={}
for i in t1:
    t1_to_index[i]=int(i)
t1_to_index

In [25]:
data['bWendu']=data.apply(lambda x:x['bWendu'].split('℃')[0],axis=1)
data['bWendu']=data.apply(lambda x:t1_to_index[x['bWendu']],axis=1)
# data['bWendu']=data.apply(lambda x:int(x['bWendu']))
data

In [26]:
t2=data['yWendu'].unique()
t2=[x.split('℃')[0] for x in t2 ]
t2

In [27]:
t2_to_index={}
for i in t2:
    t2_to_index[i]=int(i)
t2_to_index

In [28]:
data['yWendu']=data.apply(lambda x:x['yWendu'].split('℃')[0],axis=1)
data['yWendu']=data.apply(lambda x:t2_to_index[x['yWendu']],axis=1)
data

## 查看最近356次气温变化

In [29]:
data['bWendu'][-365:].plot()
data['yWendu'][-365:].plot()

## 查看最近365次空气质量变化(越高表示越差

In [34]:
data['aqi'][-365:].plot()

In [35]:

data

In [36]:
import numpy as np
# 设置5天为预测所需要的数据长度
# 设置1天为要预测的数据
seq_length=1*5
delay=1*1

data_=[]
for i in range(len(data)-seq_length-delay):
  data_.append(data.iloc[i:i+seq_length+delay])

data_=np.array([df.values for df in data_])
data_.shape

In [37]:
np.random.shuffle(data_)
x=data_[ : ,:5 ,: ]
y=data_[ : ,-1 ,4]
split_b=int(data_.shape[0]*0.8)

train_x=x[:split_b]
train_y=y[:split_b]
test_x=x[split_b:]
test_y=y[split_b:]

train_x.shape ,train_y.shape

In [38]:
test_x.shape,test_y.shape

In [39]:
mean = train_x.mean(axis=0)
std = train_x.std(axis=0)

train_x=(train_x-mean)/std
test_x=(test_x-mean)/std

In [40]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as nppip
import matplotlib.pyplot as plt
%matplotlib inline

# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [54]:
batch_size=128
with strategy.scope():
    model=keras.Sequential()
    model.add(layers.Flatten(input_shape=(train_x.shape[1:])))
    model.add(layers.Dense(32,activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='adam',
           loss='mse')
    #        metrics=['mae'])
    history=model.fit(train_x,train_y,
              batch_size=batch_size,
              epochs=10,
              validation_data=(test_x,test_y))

In [55]:
plt.plot(history.epoch,history.history['val_loss'])
plt.plot(history.epoch,history.history['loss'],color='r')

In [49]:
import tensorflow.keras.callbacks
from tensorflow.keras.callbacks import ReduceLROnPlateau
with strategy.scope():
    model=keras.Sequential()
    model.add(layers.LSTM(32,input_shape=(5,14),return_sequences=True))
    model.add(layers.LSTM(32,return_sequences=True))
    model.add(layers.LSTM(32,return_sequences=True))
    model.add(layers.LSTM(32))
    model.add(layers.Dense(1))
    # ReduceLROnPlateau
    # ReduceLROnPlateau
    lr_reduce=ReduceLROnPlateau('val_loss',patience=3,factor=0.4,min_lr=0.00001)
    # 设置在训练过程中降低学习速率，如果在3个epoch内无变化，那么学习速率就要乘以0.4，但是不能小于min_lr

    model.compile(optimizer='adam',
           loss='mse')
#            metrics=['mae'])

    history=model.fit(train_x,train_y,
              batch_size=batch_size,
              epochs=10,
              callbacks=[lr_reduce],
              validation_data=(test_x,test_y))

In [50]:
plt.plot(history.epoch,history.history['val_loss'])
plt.plot(history.epoch,history.history['loss'],color='r')

In [51]:
model.save('aqi_guangzhou_LSTM.h5')

In [52]:
model.evaluate(test_x,test_y,verbose=0)

In [56]:
batch_size=128
with strategy.scope():
    model=keras.Sequential()
    model.add(layers.Flatten(input_shape=(train_x.shape[1:])))
    model.add(layers.Dense(32,activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='adam',
           loss='mse',
           metrics=['mae'])
    history=model.fit(train_x,train_y,
              batch_size=batch_size,
              epochs=100,
              validation_data=(test_x,test_y))

In [58]:
history.history.keys()

In [59]:
plt.plot(history.epoch,history.history['val_loss'])
plt.plot(history.epoch,history.history['loss'],color='r')

# 我不知道为什么会收敛得这么这么快

In [60]:
model.save('aqi_guangzhou_dense100.h5')

In [61]:
model.evaluate(test_x,test_y,verbose=0)

In [62]:
with strategy.scope():
    model=keras.Sequential()
    model.add(layers.LSTM(32,input_shape=(5,14),return_sequences=True))
    model.add(layers.LSTM(32,return_sequences=True))
    model.add(layers.LSTM(32,return_sequences=True))
    model.add(layers.LSTM(32))
    model.add(layers.Dense(1))
    # ReduceLROnPlateau
    # ReduceLROnPlateau
    lr_reduce=ReduceLROnPlateau('val_loss',patience=7,factor=0.4,min_lr=0.0001)
    # 设置在训练过程中降低学习速率，如果在3个epoch内无变化，那么学习速率就要乘以0.4，但是不能小于min_lr

    model.compile(optimizer='adam',
           loss='mse',
           metrics=['mae'])

    history=model.fit(train_x,train_y,
              batch_size=batch_size,
              epochs=100,
              callbacks=[lr_reduce],
              validation_data=(test_x,test_y))

In [66]:
plt.plot(history.epoch,history.history['val_loss'])
plt.plot(history.epoch,history.history['loss'],color='r')

In [67]:
model.save('aqi_guangzhou_LSTM100.h5')

In [68]:
model.evaluate(test_x,test_y,verbose=0)